# Data Pre-processing

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPool1D, Dropout, Reshape
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [2]:
weather_raw_df = pd.read_csv('the_ultimate_complete_data_delet.csv',',',header=0,dtype='float64',nrows=100000)
weather_raw_df.drop(['Belfort Rain Gauge','Logger Battery Voltage','Year','Julian Day','Time'],axis=1,inplace=True)

weather_labels = weather_raw_df['Precipitation (Tipping Bucket) 3H'].copy()

weather_raw_df.drop(['Precipitation (Tipping Bucket) 3H'],axis=1,inplace=True)

weather_raw_df.head(10)

,Net Radiometer,Incoming Shortwave Radiation,Reflected Shortwave Radiation,Relative Humidity,Ambient Air Temperature,Barometric Pressure,Wind Direction,Wind Speed,Wind Speed at Rain Gauge,Precipitation (Tipping Bucket)
0,-3.076,-2.500,0.000,61.36,2.046,101.4,107.9,2.639,2.217,9.0
1,-3.080,-2.500,-0.629,61.69,1.972,101.4,109.9,2.082,1.647,9.0
2,-3.085,-2.500,0.000,62.97,1.822,101.4,97.5,1.986,1.491,9.0
3,-3.089,-2.500,0.000,62.50,1.699,101.4,91.7,2.513,1.975,9.0
4,-3.094,-2.500,0.000,63.37,1.737,101.3,92.5,3.000,2.501,9.0
5,-3.097,-2.500,0.000,65.99,1.371,101.3,95.5,2.406,1.848,9.0
6,-3.100,-2.500,0.000,67.20,1.436,101.3,94.9,2.722,2.252,9.0
7,-3.102,-1.875,0.000,65.39,1.531,101.3,92.9,2.956,2.437,9.0
8,-3.104,-2.500,0.000,65.73,1.559,101.3,97.7,3.148,2.680,9.0
9,-3.105,-2.500,0.000,65.12,1.549,101.3,99.9,3.613,3.109,9.0


In [3]:
mask = weather_raw_df['Precipitation (Tipping Bucket)'] > 0
mask2 = weather_raw_df['Precipitation (Tipping Bucket)'] <= 0
weather_raw_df.loc[mask,'Precipitation (Tipping Bucket)'] = 1
weather_raw_df.loc[mask2,'Precipitation (Tipping Bucket)'] = 0

In [4]:
i = 0
for column in weather_raw_df:
    if i < 9:
        weather_raw_df.loc[:,column] += np.absolute(weather_raw_df[column].min())
        weather_raw_df.loc[:,column] /= weather_raw_df[column].max()
        weather_raw_df.loc[:,column] -= np.mean(weather_raw_df[column])
        #weather_raw_df.loc[:,column] /= np.std(weather_raw_df[column])
    i += 1

In [5]:
mask = weather_labels > 0
mask2 = weather_labels <= 0
weather_labels.loc[mask] = 1
weather_labels.loc[mask2] = 0

# Train Test Data Split

In [6]:
x_train, x_test, y_train, y_test = train_test_split(weather_raw_df, weather_labels, test_size=0.2, random_state=69, shuffle=True)

# CNN Model

In [ ]:
model = Sequential([
     Reshape((1,10), input_shape=(10,)),
     Conv1D(32, (1), activation='relu', input_shape=(1,10)),
     Conv1D(64, (1), activation='relu'),
     MaxPool1D(1),
     Flatten(),
     Dense(128, activation='relu'),
     Dropout(0.2),
     Dense(128, activation='relu'),
     Dropout(0.2),
     Dense(128, activation='relu'),
     Dense(128, activation='relu'),
     Dense(2, activation='softmax')
])

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# info = model.fit(x=x_train, y=y_train, validation_data=(x_val,y_val), epochs=15)
info = model.fit(x=x_train, y=y_train, validation_split=0.2, epochs=3)

In [ ]:
predictions = model.predict(x_test)
predictions = np.argmax(predictions,axis=1)

In [ ]:
print(classification_report(y_test, predictions))

# SVM Model

In [12]:
from sklearn import svm

In [13]:
SVM = svm.SVC(kernel='sigmoid',gamma='auto', verbose=True)

In [14]:
SVM.fit(x_train, y_train)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=True)

In [15]:
svm_predictions = SVM.predict(x_test)
#svm_predictions = np.argmax(svm_predictions,axis=1)
svm_predictions

array([0., 0., 0., ..., 0., 0., 0.])

In [12]:
print(classification_report(y_test, svm_predictions)) #rbf gamma=scale

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     19145
         1.0       0.64      0.15      0.24       855

   micro avg       0.96      0.96      0.96     20000
   macro avg       0.80      0.57      0.61     20000
weighted avg       0.95      0.96      0.95     20000



In [11]:
print(classification_report(y_test, svm_predictions)) #rbf gamma=auto

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     19145
         1.0       0.00      0.00      0.00       855

   micro avg       0.96      0.96      0.96     20000
   macro avg       0.48      0.50      0.49     20000
weighted avg       0.92      0.96      0.94     20000



C:\Users\CAN\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
print(classification_report(y_test, svm_predictions)) #sigmoid gamma-scale

              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96     19145
         1.0       0.16      0.17      0.16       855

   micro avg       0.93      0.93      0.93     20000
   macro avg       0.56      0.56      0.56     20000
weighted avg       0.93      0.93      0.93     20000



In [16]:
print(classification_report(y_test, svm_predictions)) #sigmoid gamma-auto

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     19145
         1.0       0.00      0.00      0.00       855

   micro avg       0.96      0.96      0.96     20000
   macro avg       0.48      0.50      0.49     20000
weighted avg       0.92      0.96      0.94     20000



C:\Users\CAN\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
print(classification_report(y_test, svm_predictions)) #poly3

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     19145
         1.0       0.70      0.09      0.17       855

   micro avg       0.96      0.96      0.96     20000
   macro avg       0.83      0.55      0.57     20000
weighted avg       0.95      0.96      0.94     20000



In [26]:
print(classification_report(y_test, svm_predictions)) #poly5 C1

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98     19145
         1.0       0.66      0.24      0.35       855

   micro avg       0.96      0.96      0.96     20000
   macro avg       0.82      0.62      0.67     20000
weighted avg       0.95      0.96      0.95     20000



In [14]:
print(classification_report(y_test, svm_predictions)) #poly4 C1.5

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98     19145
         1.0       0.66      0.23      0.34       855

   micro avg       0.96      0.96      0.96     20000
   macro avg       0.81      0.61      0.66     20000
weighted avg       0.95      0.96      0.95     20000



In [18]:
print(classification_report(y_test, svm_predictions)) #poly4 C2

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98     19145
         1.0       0.66      0.23      0.34       855

   micro avg       0.96      0.96      0.96     20000
   macro avg       0.81      0.61      0.66     20000
weighted avg       0.95      0.96      0.95     20000



In [ ]:
print(classification_report(y_test, svm_predictions)) #poly6 C.75